# Importación e Intalación de Librerías

In [1]:
import csv
import pandas as pd
import sys
import numpy as np
import os
from hermetrics.levenshtein import Levenshtein
import re
import mysql.connector
from tqdm import tqdm

# Ruta de archivos

In [2]:
print('Definición de rutas de entrada y salida')

Definición de rutas de entrada y salida


In [3]:
#Rutas de entrada
path_sd = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_dd = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DOWNLOADED_DATA'))
path_ad = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))

#Rutas salida
path_db = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))
path_dwh = os.path.join(os.getcwd(), "DATAWAREHOUSE")
path_dm_oc = os.path.join(path_dwh,'datamart_oc')
path_dm_oec = os.path.join(path_dwh,'datamart_oec')
path_dm_ci = os.path.join(path_dwh,'datamart_ci')

# Funciones

In [4]:
print('Definición de Funciones')

Definición de Funciones


In [5]:
stopwords = open(os.path.join(path_db,'spanish.txt'),"r", encoding='utf-8')
stopwords = stopwords.readlines()
stopwords = [x.replace('\n','') for x in stopwords]
global puntuacion
puntuacion = '!"#$%&:;\'()*+,-./?@[\]^_`{|}~()'

In [6]:
def normalizar(text):
    text = text.lower() 
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.strip()
    text = text.split(' ')
    re_punc = re.compile('[%s]' % re.escape(puntuacion))
    stripped = [re_punc.sub(' ', w) for w in text]
    text = ' '.join(stripped)
    text = text.split(' ')
    new_text = []
    for i in range(len(text)):
        char = text[i]
        if char not in stopwords and char!=' ':
            new_text.append(text[i])
    new_text = ' '.join(new_text)
    new_text = new_text.strip()
            
    return new_text

In [7]:
def get_costos(dfp,df_info):
    
    return dfp

In [8]:
year_list = [str(x) for x in range(2022,2009,-1)]

def get_year(x):
    for year in year_list:
        if re.search('-'+year+'\s|'+year+'-|-'+year+'$',x.replace('/','-')):
            return '31-12-'+year
    return 'sin_anio'

In [9]:
def get_year_by_oc(x,y):
    if x=='sin_anio':
        if y != 'sin fecha resolución' and not pd.isna(y):
            return '31-12-'+str(y[-4:])
    else:
        return x

In [10]:
def get_perfil_price(x,y):
    if x=='perfil':
        return 250.0
    else:
        return y

In [11]:
def get_perfil_hours(x,y):
    if x=='perfil':
        return 6
    else:
        return y

In [12]:
lev = Levenshtein()

# EXTRACCIÓN DE DATOS

In [13]:
print('############ FASE DE EXTRACCIÓN DE DATOS ############')

############ FASE DE EXTRACCIÓN DE DATOS ############


## Catálogo Nacional de Cualificaciones - CNC 

- CNC para Operadores de Capacitación - CNCOC
- CNC para Organismos Evaluadores de  la Conformidad - CNCOEC
- CNC  Perfiles Inhabilitados - CNCPI

In [69]:
print('ETL CNC')

ETL CNC


### ETL CNCOC

In [70]:
df_cncoc = pd.read_csv(os.path.join(path_sd,'cncoc.csv'))
df_cncoc.insert(4, 'tipo', "oc")
df_cncoc.insert(5, 'estado', "habilitado")

In [71]:
# df_cncoc.info()

### ETL CNCOEC

In [73]:
df_cncoec = pd.read_csv(os.path.join(path_sd,'cncoec0.csv'))
df_cncoec.insert(4, 'tipo', "oec")
df_cncoec.insert(5, 'estado', "habilitado")

In [74]:
# df_cncoec.info()

### ETL CNCPI

In [75]:
df_cncpi = pd.read_csv(os.path.join(path_sd,'cncpi0.csv'))
df_cncpi.insert(1, 'competenecia_laboral', "sin competencia laboral")
df_cncpi.insert(4, 'tipo', "desconocido")
df_cncpi.insert(5, 'estado', "inhabilidato")

In [76]:
# df_cncpi.info()

 ### **CNC** - INTEGRACIÓN DE DATOS

In [77]:
df_cnc = pd.concat([df_cncoc,df_cncoec,df_cncpi],axis=0)
df_cnc = df_cnc.apply(lambda x: x.astype(str).str.lower())

In [78]:
df_cnc = df_cnc.drop_duplicates()

In [79]:
#df_cnc.tail()
#df_cnc.info()

### Guardar datos CNC

In [80]:
df_cnc.to_csv(os.path.join(path_db,'cnc.csv'),index=False)

## Operadores de Capacitación - OC

• [Operadores de Capacitación](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt25) - OC

• [OC - Suspendidos](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt125) - OCS

• [OC - Finalizaron su vigencia de Calificación](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt134) - OCF

• [OC - Cancelados](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt144) - OCC


In [81]:
print('ETL OC')

ETL OC


### ETL OC

In [82]:
df_oc = pd.read_csv(os.path.join(path_sd,'oc0.csv'))
df_oc['ruc_o_codigo'] = df_oc['ruc_o_codigo'].astype('object')
df_oc["nombre_comercial"].fillna("Sin Nombre Comercial", inplace = True) 
df_oc.insert(9, 'estado', df_oc.pop('estado'))

In [83]:
df_oc['ruc_o_codigo'] = df_oc['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)
df_oc['celular'] = df_oc['celular'].apply(lambda x: '0'+x if len(x)==9 else x)

In [84]:
#df_oc.head()
#df_oc.info()

### ETL OCS

In [85]:
df_ocs = pd.read_csv(os.path.join(path_sd,'ocs.csv'))
df_ocs['ruc_o_codigo'] = df_ocs['ruc_o_codigo'].astype('object')
df_ocs.insert(2, 'nombre_comercial', 'Sin Nombre Comercial')
df_ocs.insert(3, 'telefono', 'Sin Teléfono')
df_ocs.insert(4, 'celular', 'Sin Celular')
df_ocs.insert(5, 'correo_electronico', 'Sin Email')
df_ocs.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_ocs.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_ocs.insert(8, 'canton', 'Sin Cantón')

In [86]:
#df_ocs.head()
#df_ocs.info()

### ETL OCC

In [87]:
df_occ = pd.read_csv(os.path.join(path_sd,'occ.csv'))
df_occ['ruc_o_codigo'] = df_occ['ruc_o_codigo'].astype('str')
df_occ.insert(2, 'nombre_comercial', 'Sin Nombre Comercial')
df_occ.insert(3, 'telefono', 'Sin Teléfono')
df_occ.insert(4, 'celular', 'Sin Celular')
df_occ.insert(5, 'correo_electronico', 'Sin Email')
df_occ.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_occ.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_occ.insert(8, 'canton', 'Sin Cantón')

In [88]:
df_occ['ruc_o_codigo'] = df_occ['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [89]:
#df_occ.head()
#df_occ.info()

### ETL OCF

In [90]:
df_ocf = pd.read_csv(os.path.join(path_sd,'ocf0.csv'))
df_ocf['ruc_o_codigo'] = df_ocf['ruc_o_codigo'].astype('str')
df_ocf.insert(2, 'nombre_comercial', 'Sin Nombre Comercial')
df_ocf.insert(3, 'telefono', 'Sin Teléfono')
df_ocf.insert(4, 'celular', 'Sin Celular')
df_ocf.insert(5, 'correo_electronico', 'Sin Correo Electronico')
df_ocf.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_ocf.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_ocf.insert(8, 'canton', 'Sin Cantón')

In [91]:
df_ocf['ruc_o_codigo'] = df_ocf['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [92]:
#df_ocf.head()
#df_ocf.info()

### DATOS LIMPIOS OC

In [93]:
df_oc = pd.concat([df_oc,df_ocs,df_ocf,df_occ],axis=0)
df_oc = df_oc.apply(lambda x: x.astype(str).str.lower())
df_oc = df_oc.drop_duplicates()
df_oc = df_oc.drop_duplicates(subset='razon_social',keep='first')

In [94]:
#df_oc.head()
#df_OC.info()

### Guardar datos OC

In [95]:
df_oc.to_csv(os.path.join(path_db,'oc.csv'),index=False)

### ETL OC_CC

In [54]:
df_oc_cc = pd.read_csv(os.path.join(path_sd,'oc_cc0.csv'))
df_oc_cc = df_oc_cc.dropna(subset=['area','especialidad','curso','modalidad','carga_horaria'])
df_oc_cc = df_oc_cc.apply(lambda x: x.astype(str).str.lower())
df_oc_cc['carga_horaria'] = df_oc_cc['carga_horaria'].astype('int64')
df_oc_cc = df_oc_cc.drop_duplicates()

In [55]:
#df_oc_cc.head()
#df_oc_cc.info()

### Guardar datos OC_CC

In [56]:
df_oc_cc.to_csv(os.path.join(path_db,'oc_cc.csv'),index=False)

### ETL OC_CL

In [99]:
df_oc_cl = pd.read_csv(os.path.join(path_sd,'oc_cl0.csv'))
df_oc_cl = df_oc_cl.apply(lambda x: x.astype(str).str.lower())
df_oc_cl = df_oc_cl.drop_duplicates()

In [100]:
#df_oc_cl.head()
#df_oc_cl.info()

### Guardar datos OC_CL

In [101]:
df_oc_cl.to_csv(os.path.join(path_db,'oc_cl.csv'),index=False)

## Personas Capacitadas por OC - PCOC

• [Búsqueda Personas Capacitadas](http://portal.trabajo.gob.ec/setec-portal-web/pages/personasCapacitadasOperadores.jsf#j_idt24:j_idt25) - PCOC

In [102]:
print('ETL PCOC')

ETL PCOC


In [103]:
df_pcoc = pd.read_csv(os.path.join(path_sd,'pcoc.csv'))
df_pcoc["numero_documento"].fillna("Sin Número Documento", inplace = True)
df_pcoc["apellidos_nombres"].fillna("Sin Apellidos y Nombres", inplace = True)
df_pcoc["numero_horas"].fillna(0, inplace = True)
df_pcoc["numero_horas"]=df_pcoc["numero_horas"].astype('int64')
df_pcoc["razon_social_oc"].fillna("Sin Razón Social OC", inplace = True)
df_pcoc["nombre_comercial_oc"].fillna("Sin Nombre Comercial OC", inplace = True)
df_pcoc = df_pcoc.apply(lambda x: x.astype(str).str.lower())
df_pcoc["numero_horas"]=df_pcoc["numero_horas"].astype('int64')
df_pcoc = df_pcoc.drop_duplicates()    

In [104]:
#df_pcoc.head()
#df_pcoc.info()

In [105]:
df_pcoc.to_csv(os.path.join(path_db,'pcoc.csv'),index=False)

## Organismos Evaluadores de  la Conformidad - OEC

• [Organismos Evaluadores de la Conformidad](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt25) - OEC

• [OEC - Suspendidos](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt103) - OECS

• [OEC - Finalizaron su vigencia de Reconocimiento](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt112) - OECF

• [OEC - Cancelados](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt122) - OECC

In [106]:
print('ETL OEC')

ETL OEC


### ETL OEC

In [107]:
df_oec = pd.read_csv(os.path.join(path_sd,'oec0.csv'))
df_oec['celular'] = df_oec['celular'].astype('object')
df_oec = df_oec.apply(lambda x: x.astype(str).str.lower())
df_oec['ruc_o_codigo'] = df_oec['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)
df_oec['celular'] = df_oec['celular'].apply(lambda x: '0'+x if len(x)==9 else x)
df_oec = df_oec.drop_duplicates()

In [108]:
#df_oec.head()
#df_oec.info()

### ETL OECS

In [109]:
df_oecs = pd.read_csv(os.path.join(path_sd,'oecs.csv'))
df_oecs.insert(2, 'direccion', 'Sin Dirección')
df_oecs.insert(3, 'telefono', 'Sin Teléfono')
df_oecs.insert(4, 'celular', 'Sin Celular')
df_oecs.insert(5, 'correo_electronico', 'Sin Email')
df_oecs.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_oecs.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_oecs.insert(8, 'canton', 'Sin Cantón')
df_oecs = df_oecs.apply(lambda x: x.astype(str).str.lower())

In [110]:
df_oecs['ruc_o_codigo'] = df_oecs['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [111]:
#df_oecs.head()
#df_oecs.info()

### ETL OECF

In [112]:
df_oecf = pd.read_csv(os.path.join(path_sd,'oecf0.csv'))
df_oecf.insert(2, 'direccion', 'Sin Dirección')
df_oecf.insert(3, 'telefono', 'Sin Teléfono')
df_oecf.insert(4, 'celular', 'Sin Celular')
df_oecf.insert(5, 'correo_electronico', 'Sin Email')
df_oecf.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_oecf.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_oecf.insert(8, 'canton', 'Sin Cantón')
df_oecf = df_oecf.apply(lambda x: x.astype(str).str.lower())
df_oecs['ruc_o_codigo'] = df_oecs['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [113]:
#df_oecf.head()
#df_oecf.info()

### ETL OECC

In [114]:
df_oecc = pd.read_csv(os.path.join(path_sd,'oecc.csv'))
df_oecc.insert(2, 'direccion', 'Sin Dirección')
df_oecc.insert(3, 'telefono', 'Sin Teléfono')
df_oecc.insert(4, 'celular', 'Sin Celular')
df_oecc.insert(5, 'correo_electronico', 'Sin Email')
df_oecc.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_oecc.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_oecc.insert(8, 'canton', 'Sin Cantón')                               
df_oecc = df_oecc.apply(lambda x: x.astype(str).str.lower())
df_oecc['ruc_o_codigo'] = df_oecc['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [115]:
#df_oecc.head()
#df_oecc.info()

### DATOS LIMPIOS OEC

In [116]:
df_oec = pd.concat([df_oec,df_oecs,df_oecf,df_oecc],axis=0)
df_oec = df_oec.drop_duplicates()
df_oec = df_oec.drop_duplicates(subset='razon_social',keep='first')

In [117]:
#df_OEC.head()
#df_OEC.info()

### Guardar datos OEC

In [118]:
df_oec.to_csv(os.path.join(path_db,'oec.csv'),index=False)

### ETL OEC_DR

In [119]:
df_oec_dr = pd.read_csv(os.path.join(path_sd,'oec_dr0.csv'))
df_oec_dr["perfil"].fillna("Sin Perfil", inplace = True)
df_oec_dr["esquema_de_certificacion"].fillna("Sin Esquema de Certificación", inplace = True)
df_oec_dr = df_oec_dr.apply(lambda x: x.astype(str).str.lower())
df_oec_dr['ruc_o_codigo'] = df_oec_dr['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)
df_oec_dr = df_oec_dr.drop_duplicates()

In [120]:
#df_oec_dr.head()
#df_oec_dr.info()

### Guardar datos OEC_DR

In [121]:
df_oec_dr.to_csv(os.path.join(path_db,'oec_dr.csv'),index=False)

## Personas Certificadas por OEC - PCOEC

• [Búsqueda Personas Certificadas](http://portal.trabajo.gob.ec/setec-portal-web/pages/legitimidadCertificacion.jsf#j_idt24:j_idt25) - PCOEC

In [122]:
print('ETL PCOEC')

ETL PCOEC


In [123]:
df_pcoec = pd.read_csv(os.path.join(path_sd,'pcoec.csv'))

df_pcoec = df_pcoec.apply(lambda x: x.astype(str).str.lower())
df_pcoec['ruc_o_codigo'] = df_pcoec['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)
df_pcoec['celular'] = df_pcoec['celular'].apply(lambda x: '0'+x if len(x)==9 else x)
df_pcoec = df_pcoec.drop_duplicates()

In [124]:
#df_pcoec.head()
#df_pcoec.info()

### Guardar datos PCOEC

In [125]:
df_pcoec.to_csv(os.path.join(path_db,'pcoec.csv'),index=False)

## Capacitadores Independientes - CI

• [Búsqueda de Capacitadores Independientes](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacionIndependientes.jsf#j_idt24:j_idt25) - CI

In [126]:
print('ETL CI')

ETL CI


### ETL CI

In [252]:
df_ci = pd.read_csv(os.path.join(path_sd,'ci0.csv'))
df_ci = df_ci.apply(lambda x: x.astype(str).str.lower())
df_ci['numero_documento'] = df_ci['numero_documento'].apply(lambda x: '0'+x if len(x)==9 else x)
df_ci['celular'] = df_ci['celular'].apply(lambda x: '0'+x if len(x)==9 else x)
df_ci = df_ci.drop_duplicates()
df_ci = df_ci.rename(columns={'apellidos_nombres':'razon_social'})
df_ci = df_ci.rename(columns={'codigo_resolucion':'fecha_resolucion'})

In [26]:
#df_ci.head()
#df_ci.info()

### Guardar datos CI

In [27]:
df_ci.to_csv(os.path.join(path_db,'ci.csv'),index=False)

### ETL CI_LCA

In [130]:
df_ci_lca =pd.read_csv(os.path.join(path_sd,'ci_lca0.csv'))
df_ci_lca["carga_horaria"].fillna(0, inplace = True)
df_ci_lca["carga_horaria"]=df_ci_lca["carga_horaria"].astype('int64')
df_ci_lca = df_ci_lca.apply(lambda x: x.astype(str).str.lower())
df_ci_lca["carga_horaria"]=df_ci_lca["carga_horaria"].astype('int64')
df_ci_lca = df_ci_lca.drop_duplicates()

In [131]:
#df_ci_lca.head()
#df_ci_lca.info()

### Guardar datos CI_LCA

In [132]:
df_ci_lca.to_csv(os.path.join(path_db,'ci_lca.csv'),index=False)

## Personas Capacitadas por CI - PCCI

• [Búsqueda de Personas Capacitadas](http://portal.trabajo.gob.ec/setec-portal-web/pages/personasCapacitadas.jsf#j_idt24:j_idt25) - PCCI

In [133]:
print('ETL PCCI')

ETL PCCI


In [259]:
df_pcci =pd.read_csv(os.path.join(path_sd,'pcci.csv'))
df_pcci = df_pcci.apply(lambda x: x.astype(str).str.lower())
df_pcci["numero_horas"]=df_pcci["numero_horas"].astype('int64')
df_pcci = df_pcci.drop_duplicates()

In [261]:
#df_pcci.head()
#df_pcci.info()

### Guardar datos PCCI

In [262]:
df_pcci.to_csv(os.path.join(path_db,'pcci.csv'),index=False)

# TRANSFORMACIÓN DE DATOS

In [137]:
print('############ FASE DE TRANSFORMACIÓN DE DATOS ###########')

############ FASE DE TRANSFORMACIÓN DE DATOS ###########


 ## Tabla Ubicacion

In [18]:
df_ubicacion = pd.read_csv(os.path.join(path_db, 'ubicacion.csv'))

# DIMENSION Razon Social (OC, OEC y CI)

In [14]:
print('DIMENSION Razon Social (OC, OEC y CI)')

DIMENSION Razon Social (OC, OEC y CI)


In [15]:
df_oc = pd.read_csv(os.path.join(path_db,'oc.csv'))
df_oec = pd.read_csv(os.path.join(path_db,'oec.csv'))
df_ci = pd.read_csv(os.path.join(path_db,'ci.csv'))
df_pcoc = pd.read_csv(os.path.join(path_db,'pcoc.csv'))
df_pcoec = pd.read_csv(os.path.join(path_db,'pcoec.csv'))
df_pcci = pd.read_csv(os.path.join(path_db,'pcci.csv'))

In [19]:
df_oc_ug = pd.read_csv(os.path.join(path_db,'oc_dl_provincia_canton.csv'))
df_oc_ug['estado']='desconocido'
df_razon_social_oc = pd.concat([df_oc[['razon_social','estado','fecha_resolucion','canton']],df_oc_ug[['razon_social','estado','canton']]],axis=0)
df_razon_social_oc.insert(2,'tipo_razon_social','operador de capacitación')
df_razon_social_oc = df_razon_social_oc.drop_duplicates(subset=['razon_social','canton'],keep='first')
df_oec_ug = pd.read_csv(os.path.join(path_db,'oec_dl_provincia_canton.csv'))
df_oec_ug['estado'] = 'desconocido'
df_razon_social_oec = pd.concat([df_oec[['razon_social','estado','fecha_resolucion','canton']],df_oec_ug[['razon_social','estado','canton']]])
df_razon_social_oec.insert(2,'tipo_razon_social','organismo evaluador de la conformidad')
df_razon_social_oec = df_razon_social_oec.drop_duplicates(subset=['razon_social','canton'], keep='first')
df_ci_ug = pd.read_csv(os.path.join(path_db,'ci_dl_provincia_canton.csv'))
df_razon_social_ci = pd.merge(df_ci,df_ci_ug,on=['razon_social'],how="inner")
df_razon_social_ci.insert(1,'estado','calificado')
df_razon_social_ci = df_razon_social_ci[['razon_social','estado','fecha_resolucion','canton']]
df_razon_social_ci.insert(2,'tipo_razon_social','capacitador independiente')
df_razon_social_ci = df_razon_social_ci.drop_duplicates(subset=['razon_social','canton'],keep='first')
df_razon_social = pd.concat([df_razon_social_oc,df_razon_social_oec,df_razon_social_ci],axis=0)
df_razon_social = pd.merge(df_razon_social,df_ubicacion,on=['canton'],how='left')
df_razon_social.drop(['id_ubicacion'],axis=1,inplace=True)
df_razon_social = df_razon_social.fillna({'provincia':'sin provincia'})
df_razon_social = df_razon_social.fillna({'fecha_resolucion':'sin fecha resolución'})
df_razon_social['estado']=pd.Categorical(df_razon_social['estado'],categories=['calificado', 'habilitado', 'finalizo vigencia de calificacion','finalizo vigencia de reconocimiento','cancelado definitivamente','suspendido','desconocido'])
df_razon_social=df_razon_social.sort_values('estado')
df_razon_social = df_razon_social.drop_duplicates(subset=['razon_social','tipo_razon_social'],keep='first')
df_razon_social=df_razon_social.sort_values(by=['razon_social','tipo_razon_social'])
df_razon_social = df_razon_social.reset_index().rename_axis('id_razon_social').reset_index().astype('str').drop(['index'],axis=1)

In [20]:
#df_razon_social.head()
#df_razon_social.info()

# Dimensión Curso-Perfil (OC, OEC, CI)

In [325]:
df_oc_cl = pd.read_csv(os.path.join(path_db,'oc_cl.csv')) # Comentar en produccion
df_oc_cl = df_oc_cl.drop(['ruc_o_codigo'],axis=1)
df_oc_cl_2 = pd.read_csv(os.path.join(path_db,'oc_dl_familia_sector_perfil.csv'))
df_oc_cl = pd.concat([df_oc_cl,df_oc_cl_2],axis=0)
df_oc_cl = df_oc_cl.drop_duplicates()
df_oc_cl['modalidad'] = 'desconocida'
df_oc_cl['carga_horaria'] = 6
df_oc_cl['tipo'] = 'perfil'
df_oc_cl['costo'] =250.0
df_oc_cl['tipo_razon_social'] ='operador de capacitación'

In [326]:
df_oc_cc = pd.read_csv(os.path.join(path_db,'oc_cc.csv')) # Comentar en produccion
df_oc_cc = df_oc_cc.drop(['documento'],axis=1)
df_oc_cc['tipo'] = 'curso'
df_oc_cc['costo'] =0.0
df_oc_cc['tipo_razon_social'] ='operador de capacitación'

In [327]:
df_perfil = pd.read_csv(os.path.join(path_db,'oec_dl_familia_sector_perfil.csv'))
df_perfil['modalidad'] = 'desconocida'
df_perfil['carga_horaria'] = 6
df_perfil['tipo'] = 'perfil'
df_perfil['costo'] = 250.0
df_perfil['tipo_razon_social'] ='organismo evaluador de la conformidad'

In [328]:
df_ci_lca = pd.read_csv(os.path.join(path_db,'ci_lca.csv')) # Comentar en produccion
df_ci_lca = df_ci_lca[['apellidos_nombres','area','especialidad','curso','carga_horaria']]
df_ci_lca.insert(4,'modalidad', 'desconocida')
df_ci_lca['tipo'] = 'curso'
df_ci_lca['costo'] = 0.0
df_ci_lca['tipo_razon_social'] ='capacitador independiente'

In [375]:
data_curso_perfil = np.concatenate((df_oc_cc.values,df_oc_cl.values,df_perfil.values,df_ci_lca.values), axis=0)
df_curso_perfil = pd.DataFrame(data=data_curso_perfil[:,:-1],columns=['razon_social','area_familia','especialidad_sector','curso_perfil','modalidad','carga_horaria','tipo','costo'])
df_curso_perfil.insert(0,'tipo',df_curso_perfil.pop('tipo'))
df_curso_perfil = df_curso_perfil.apply(lambda x: x.astype(str).str.lower())
df_curso_perfil['carga_horaria'] = df_curso_perfil['carga_horaria'].astype('int64')
data_rs_canton = df_razon_social.groupby(['razon_social','canton']).count().reset_index()[['razon_social','canton']]
df_curso_perfil = pd.merge(df_curso_perfil,data_rs_canton,on='razon_social',how="left")
df_curso_perfil['canton'] = df_curso_perfil['canton'].fillna('sin_canton')
df_curso_perfil = df_curso_perfil.drop_duplicates()
df_curso_perfil = df_curso_perfil.rename_axis('id_curso_perfil').reset_index()

In [376]:
df_costos_estimados_cursos = pd.read_csv(os.path.join(path_db,'costos_estimados_cursos_perfiles.csv'))
df_curso_perfil = pd.merge(df_curso_perfil,df_costos_estimados_cursos,on='id_curso_perfil',how='inner')

In [377]:
df_curso_perfil['costo']=df_curso_perfil['carga_horaria']*df_curso_perfil['precio_estimado_por_hora']

In [378]:
df_curso_perfil['costo']=df_curso_perfil.apply(lambda x: get_perfil_price(x['tipo'],x['costo']),axis=1)

In [379]:
df_curso_perfil.drop('precio_estimado_por_hora',axis=1,inplace=True)

In [380]:
df_curso_perfil.to_csv(os.path.join(path_dwh,'dim_curso_perfil.csv'),index=False)

In [381]:
df_info_cursos_edu = pd.read_csv(os.path.join(path_db,'costos_cursos_educacion_continua.csv'))

# Dimension Fecha (OC, OEC, CI)

In [406]:
df_pcoc = pd.read_csv(os.path.join(path_db,'pcoc.csv'))
df_pcoc['fecha'] = df_pcoc['numero_certificado'].apply(get_year)
df_pcoc = pd.merge(df_pcoc,df_razon_social_oc[['razon_social','fecha_resolucion']],left_on=['razon_social_oc'],right_on=['razon_social'],how="left") 
df_pcoc['fecha'] = df_pcoc.apply(lambda x: get_year_by_oc(x['fecha'], x['fecha_resolucion']), axis=1)
df_fecha_oc = df_pcoc[['fecha']].groupby(['fecha']).count().reset_index()

In [407]:
df_pcoec = pd.read_csv(os.path.join(path_db,'pcoec.csv'))
df_fecha_oec = df_pcoec[['fecha_certificacion']].groupby(['fecha_certificacion']).count().reset_index()
df_fecha_oec = df_fecha_oec.rename(columns={'fecha_certificacion':'fecha'})

In [408]:
df_ci = pd.read_csv(os.path.join(path_db,'ci.csv'))
df_ci['fecha'] = df_ci['fecha_resolucion'].apply(get_year)
df_fecha_ci = pd.DataFrame(data=df_ci['fecha'].unique(),columns=['fecha'])

In [412]:
df_fecha = pd.concat([df_fecha_oc,df_fecha_oec,df_fecha_ci],axis=0)
df_fecha = df_fecha.drop_duplicates()
#df_fecha = df_fecha.reset_index().drop(['index'], axis=1)
data_fecha = df_fecha['fecha'].str.rsplit("-", expand=True)
df_fecha = pd.concat([df_fecha,data_fecha],axis=1)
df_fecha = df_fecha.rename(columns={0:'dia',1:'mes',2:'anio'})
df_fecha = df_fecha.rename_axis('id_fecha').reset_index()

In [415]:
df_fecha.to_csv(os.path.join(path_dwh,'dim_fecha.csv'),index=False)

In [112]:
df_curso_perfil['carga_horaria'] = df_curso_perfil['carga_horaria'].astype('int64')
df_curso_perfil['costo'] = df_curso_perfil['costo'].astype('float64')

# Hechos OC

In [106]:
print('Datamart OC')

Datamart OC


### Dimensión Razon Social OEC

In [125]:
df_oc_num_conv= pd.read_csv(os.path.join(os.getcwd(),'pcoc_num_conv.csv'))[['razon_social', 'numero_convocados']]

In [223]:
df_razon_social_oc = df_razon_social[df_razon_social['tipo_razon_social']=='operador de capacitación']

## Hechos OC por Curso Perfil

In [382]:
df_fact_oc_cp = df_pcoc.groupby(['razon_social_oc','nombre_curso_perfil','fecha']).count().reset_index().iloc[:,:4]
df_fact_oc_cp = df_fact_oc_cp.rename(columns={'numero_documento':'num_cap_cer'})
df_fact_oc_cp = pd.merge(df_fact_oc_cp,df_razon_social_oc[['id_razon_social','razon_social']],left_on=['razon_social_oc'],right_on=['razon_social'],how="inner")
df_fact_oc_cp = pd.merge(df_fact_oc_cp,df_curso_perfil,left_on=['razon_social_oc','nombre_curso_perfil'],right_on=['razon_social','curso_perfil'],how="inner")
df_fact_oc_cp = pd.merge(df_fact_oc_cp,df_fecha,left_on=['fecha'],right_on=['fecha'],how="inner")
df_fact_oc_cp = pd.merge(df_fact_oc_cp,df_oc_num_conv,left_on='razon_social_oc',right_on='razon_social',how='left')
df_fact_oc_cp['num_cap_cer'] = df_fact_oc_cp['num_cap_cer'].astype('float64')
df_fact_oc_cp['total_horas'] = (df_fact_oc_cp['num_cap_cer'] * df_fact_oc_cp['carga_horaria']).astype('float64')
df_fact_oc_cp['total_ganancias'] = (df_fact_oc_cp['num_cap_cer'] * df_fact_oc_cp['costo']).astype('float64')
df_fact_oc_cp['porcentaje_asistencia'] = ((df_fact_oc_cp['num_cap_cer']/df_fact_oc_cp['numero_convocados'])*100).astype('float64')
df_fact_oc_cp.fillna({'porcentaje_asistencia':0},inplace=True)
df_fact_oc_cp = df_fact_oc_cp.rename_axis('id_fact_oc_cp').reset_index()
df_fact_oc_cp = df_fact_oc_cp[['id_fact_oc_cp','id_razon_social','id_curso_perfil','id_fecha','num_cap_cer','total_horas','total_ganancias','porcentaje_asistencia']]

In [383]:
#df_fact_oc_cp.head()
#df_fact_oc_cp.info()

## Hechos OC por Razon Social

In [384]:
df_total_cap_oc = df_fact_oc_cp.groupby(['id_razon_social','id_fecha']).sum().reset_index()
df_total_cur_oc = df_fact_oc_cp.groupby(['id_razon_social','id_fecha']).count().reset_index().iloc[:,:3]
df_total_cur_oc = df_total_cur_oc.rename(columns={'id_fact_oc_cp':'total_cursos'})
df_fact_oc_rs = pd.merge(df_total_cap_oc,df_total_cur_oc,on=['id_razon_social','id_fecha'],how='inner')
df_fact_oc_rs.drop(['id_fact_oc_cp'],axis=1,inplace=True)
df_fact_oc_rs.rename(columns={'num_cap_cer':'volumen_cap_cer','total_horas':'volumen_horas','total_ganancias':'volumen_ganancias','porcentaje_asistencia':'volumen_porcentaje_asistencia'},inplace=True)
df_fact_oc_rs = df_fact_oc_rs.rename_axis('id_fact_oc_rs').reset_index()

In [385]:
#df_fact_oc_rs.head()
#df_fact_oc_rs.info()

## Guardar Hechos OC

In [386]:
df_fact_oc_cp.to_csv(os.path.join(path_dwh,'fact_oc_cp.csv'),index=False)
df_fact_oc_rs.to_csv(os.path.join(path_dwh,'fact_oc_rs.csv'),index=False)

# Dataframe OEC

### Dimensión Razon Social OEC

In [387]:
df_razon_social_oec = df_razon_social[df_razon_social['tipo_razon_social']=='organismo evaluador de la conformidad']

In [388]:
df_oec_num_conv= pd.read_csv(os.path.join(os.getcwd(),'pcoec_num_conv.csv'),encoding = "ISO-8859-1")[['razon_social', 'numero_convocados']]

## Hechos OEC por Perfil

In [389]:
df_pcoec = pd.read_csv(os.path.join(path_db,'pcoec.csv'))
df_fact_oec_cp = df_pcoec.groupby(['razon_social','perfil','fecha_certificacion']).count().reset_index().iloc[:,:4]
df_fact_oec_cp = df_fact_oec_cp.rename(columns={'apellidos_nombres':'num_cer'})
df_fact_oec_cp = pd.merge(df_fact_oec_cp,df_razon_social_oec[['id_razon_social','razon_social']],on='razon_social',how="inner")
df_fact_oec_cp = pd.merge(df_fact_oec_cp,df_curso_perfil,left_on=['razon_social','perfil'],right_on=['razon_social','curso_perfil'],how="inner")
df_fact_oec_cp = pd.merge(df_fact_oec_cp,df_fecha,left_on=['fecha_certificacion'],right_on=['fecha'],how="inner")
df_fact_oec_cp = pd.merge(df_fact_oec_cp,df_oec_num_conv,left_on='razon_social',right_on='razon_social',how='left')
df_fact_oec_cp['num_cer'] = df_fact_oec_cp['num_cer'].astype('float64')
df_fact_oec_cp['total_horas'] = (df_fact_oec_cp['num_cer'] * df_fact_oec_cp['carga_horaria']).astype('float64')
df_fact_oec_cp['total_ganancias'] = (df_fact_oec_cp['num_cer'] * df_fact_oec_cp['costo']).astype('float64')
df_fact_oec_cp['porcentaje_asistencia'] = ((df_fact_oec_cp['num_cer']/df_fact_oec_cp['numero_convocados'])*100).astype('float64')
df_fact_oec_cp.fillna({'porcentaje_asistencia':0},inplace=True)
df_fact_oec_cp = df_fact_oec_cp.rename_axis('id_fact_oec_cp').reset_index()
df_fact_oec_cp = df_fact_oec_cp[['id_fact_oec_cp','id_razon_social','id_curso_perfil','id_fecha','num_cer','total_horas','total_ganancias','porcentaje_asistencia']]

In [390]:
#df_fact_oec_cp.head()
#df_fact_oec_cp.info()

## Hechos OEC por Razon Social

In [391]:
df_total_cer_oec = df_fact_oec_cp.groupby(['id_razon_social','id_fecha']).sum().reset_index()
df_total_per_oec = df_fact_oec_cp.groupby(['id_razon_social','id_fecha']).count().reset_index().iloc[:,:3]
df_total_per_oec = df_total_per_oec.rename(columns={'id_fact_oec_cp':'total_perfiles'})
df_fact_oec_rs = pd.merge(df_total_cer_oec,df_total_per_oec,on=['id_razon_social','id_fecha'],how='inner')
df_fact_oec_rs.drop(['id_fact_oec_cp'],axis=1,inplace=True)
df_fact_oec_rs.rename(columns={'num_cer':'volumen_cer','total_horas':'volumen_horas','total_ganancias':'volumen_ganancias','porcentaje_asistencia':'volumen_porcentaje_asistencia'},inplace=True)
df_fact_oec_rs = df_fact_oec_rs.rename_axis('id_fact_oec_rs').reset_index()

In [392]:
#df_fact_oec_rs.head()
#df_fact_oec_rs.info()

## Guardar Hechos OEC

In [393]:
df_fact_oec_cp.to_csv(os.path.join(path_dwh,'fact_oec_cp.csv'),index=False)
df_fact_oec_rs.to_csv(os.path.join(path_dwh,'fact_oec_rs.csv'),index=False)

# Datamart CI

In [394]:
print('Datamart CI')

Datamart CI


## Dimensión Razon Social CI

In [395]:
df_razon_social_ci = df_razon_social[df_razon_social['tipo_razon_social']=='capacitador independiente']

## Hechos CI

In [396]:
df_pcci = pd.read_csv(os.path.join(path_db,'pcci.csv'))
df_razon_social_ci = df_razon_social[df_razon_social['tipo_razon_social']=='capacitador independiente']
df_razon_social_ci['fecha'] = df_razon_social_ci['fecha_resolucion'].apply(get_year)
df_fact_ci_cp = pd.merge(df_pcci,df_razon_social_ci[['razon_social','fecha']],on='razon_social',how='inner')
df_fact_ci_cp = df_fact_ci_cp.groupby(['razon_social','curso','fecha']).count().reset_index().iloc[:,:4]
df_fact_ci_cp = df_fact_ci_cp.rename(columns={'ruc_o_codigo':'num_cap'})
df_fact_ci_cp = pd.merge(df_fact_ci_cp,df_razon_social_ci[['id_razon_social','razon_social']],on='razon_social',how="inner")
df_fact_ci_cp = pd.merge(df_fact_ci_cp,df_curso_perfil,left_on=['razon_social','curso'],right_on=['razon_social','curso_perfil'],how="inner")
df_fact_ci_cp = pd.merge(df_fact_ci_cp,df_fecha,on='fecha',how="inner")
df_fact_ci_cp['num_cap'] = df_fact_ci_cp['num_cap'].astype('float64')
df_fact_ci_cp['total_horas'] = (df_fact_ci_cp['num_cap'] * df_fact_ci_cp['carga_horaria']).astype('float64')
df_fact_ci_cp['total_ganancias'] = (df_fact_ci_cp['num_cap'] * df_fact_ci_cp['costo']).astype('float64')
df_fact_ci_cp = df_fact_ci_cp.rename_axis('id_fact_ci_cp').reset_index()
df_fact_ci_cp = df_fact_ci_cp[['id_fact_ci_cp','id_razon_social','id_curso_perfil','id_fecha','num_cap','total_horas','total_ganancias']]

C:\Users\sjime\AppData\Local\Temp\ipykernel_18552\2977254387.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_razon_social_ci['fecha'] = df_razon_social_ci['fecha_resolucion'].apply(get_year)


In [397]:
#df_fact_ci_cp.head()
#df_fact_ci_cp.info()

In [398]:
df_total_cap_ci = df_fact_ci_cp.groupby(['id_razon_social','id_fecha']).sum().reset_index()
df_total_cur_ci = df_fact_ci_cp.groupby(['id_razon_social','id_fecha']).count().reset_index().iloc[:,:3]
df_total_cur_ci = df_total_cur_ci.rename(columns={'id_fact_ci_cp':'total_cursos'})
df_fact_ci_rs = pd.merge(df_total_cap_ci,df_total_cur_ci,on=['id_razon_social','id_fecha'],how='inner')
df_fact_ci_rs.drop(['id_fact_ci_cp'],axis=1,inplace=True)
df_fact_ci_rs.rename(columns={'num_cap':'volumen_cap','total_horas':'volumen_horas','total_ganancias':'volumen_ganancias'},inplace=True)
df_fact_ci_rs = df_fact_ci_rs.rename_axis('id_fact_ci_rs').reset_index()

In [399]:
#df_fact_ci_rs.head()
#df_fact_ci_rs.info()

### Guardar Datamart

In [403]:
df_fact_ci_cp.to_csv(os.path.join(path_dwh,'fact_ci_cp.csv'),index=False)
df_fact_ci_rs.to_csv(os.path.join(path_dwh,'fact_ci_rs.csv'),index=False)

## Guardar Curso_Perfil

In [404]:
df_curso_perfil.drop(['razon_social','canton'],axis=1,inplace=True)

In [405]:
df_curso_perfil.to_csv(os.path.join(path_dwh,'dim_curso_perfil.csv'),index=False)

# CARGA DE DATOS

# Conexión con MySQL

In [203]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='KappaTao1233'
)

mycursor = mydb.cursor()

# Creación de la base de datos

In [204]:
mycursor.execute("DROP DATABASE IF EXISTS dwh_setec")
mycursor.execute("CREATE DATABASE dwh_setec")

In [205]:
mycursor.execute("USE dwh_setec")

# DIMENSIÓN RAZÓN SOCIAL

In [206]:
#os.listdir(path_dwh)

In [207]:
df_razon_social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2257 entries, 0 to 2256
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_razon_social    2257 non-null   object
 1   razon_social       2257 non-null   object
 2   estado             2257 non-null   object
 3   tipo_razon_social  2257 non-null   object
 4   canton             2257 non-null   object
 5   provincia          2257 non-null   object
dtypes: object(6)
memory usage: 105.9+ KB


In [208]:
mycursor.execute("DROP TABLE IF EXISTS dim_razon_social")
mycursor.execute('''CREATE TABLE dim_razon_social (
id_razon_social VARCHAR(8) PRIMARY KEY,
razon_social VARCHAR(255) NOT NULL,
tipo_razon_social VARCHAR(255) NOT NULL,
estado VARCHAR(63) NOT NULL,
canton VARCHAR(63) NOT NULL,
provincia VARCHAR(63) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_razon_social ( id_razon_social, razon_social,tipo_razon_social, estado, canton, provincia) VALUES (%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_razon_social.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 2257/2257 [04:27<00:00,  8.44it/s]


# DATAMART OC

In [209]:
#os.listdir(path_dm_oc)

## Tabla dim_curso_perfil_oc

In [210]:
df_cur_per = pd.read_csv(os.path.join(path_dm_oc,'dim_curso_perfil.csv'))
df_cur_per['id_curso_perfil'] = df_cur_per['id_curso_perfil'].astype(str)
df_cur_per = df_cur_per.fillna({'costo':0})

In [211]:
mycursor.execute("DROP TABLE IF EXISTS dim_curso_perfil_oc")
mycursor.execute('''CREATE TABLE dim_curso_perfil_oc (
id_curso_perfil VARCHAR(4) PRIMARY KEY,
tipo VARCHAR(63) NOT NULL,
area_familia VARCHAR(255) NOT NULL,
especialidad_sector VARCHAR(255) NOT NULL,
curso_perfil VARCHAR(255) NOT NULL,
modalidad VARCHAR(255) NOT NULL,
carga_horaria FLOAT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_curso_perfil_oc ( id_curso_perfil, tipo, area_familia, especialidad_sector, curso_perfil, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_cur_per.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 5103/5103 [21:48<00:00,  3.90it/s]


## Tabla dim_fecha_oc

In [212]:
df_fecha_oc = pd.read_csv(os.path.join(path_dm_oc,'dim_fecha.csv'))
df_fecha_oc['id_fecha'] = df_fecha_oc['id_fecha'].astype(str)
df_fecha_oc['anio'] = df_fecha_oc['anio'].astype(str)

In [213]:
mycursor.execute("DROP TABLE IF EXISTS dim_fecha_oc")
mycursor.execute('''CREATE TABLE dim_fecha_oc (
id_fecha VARCHAR(4) PRIMARY KEY,
anio VARCHAR(4) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha_oc ( id_fecha, anio) VALUES (%s,%s)'
for row in tqdm(df_fecha_oc.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 20.27it/s]


## Tabla fact_oc

In [214]:
df_fact_oc = pd.read_csv(os.path.join(path_dm_oc,'fact_oc.csv'))
df_fact_oc['id_razon_social'] = df_fact_oc['id_razon_social'].astype(str)
df_fact_oc['id_curso_perfil'] = df_fact_oc['id_curso_perfil'].astype(str)
df_fact_oc['id_fecha'] = df_fact_oc['id_fecha'].astype(str)

In [438]:
#df_fact_oc.info()

In [439]:
mycursor.execute("DROP TABLE IF EXISTS fact_oc")
mycursor.execute('''CREATE TABLE fact_oc (
id_razon_social VARCHAR(4) NOT NULL,
id_curso_perfil VARCHAR(4) NOT NULL,
id_fecha VARCHAR(4) NOT NULL,
num_cap_cer INT NOT NULL,
total_cursos INT NOT NULL,
volumen_capacitados INT NOT NULL,
total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oc (id_razon_social, id_curso_perfil, id_fecha, num_cap_cer, total_cursos, volumen_capacitados, total_horas, total_ganancias) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(df_fact_oc.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████| 1580/1580 [01:08<00:00, 23.02it/s]


## CREAR CLAVES FORANEAS OC

In [440]:
mycursor.execute(''' 
ALTER TABLE fact_oc
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha_oc(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_oc
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_oc
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil_oc(id_curso_perfil)
''')


# DATAMART OEC

In [441]:
#os.listdir(path_dm_oec)

In [442]:
df_oec = pd.read_csv(os.path.join(path_dm_oec,'dim_razon_social_oec.csv'))
df_oec['id_oec'] = df_oec['id_oec'].astype(str)

NameError: name 'path_dmoec' is not defined

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_razon_social_oec")
mycursor.execute('''CREATE TABLE dim_razon_social_oec (
id_oec VARCHAR(4) PRIMARY KEY,
razon_social VARCHAR(255) NOT NULL,
estado VARCHAR(63) NOT NULL,
canton VARCHAR(63) NOT NULL,
provincia VARCHAR(63) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_razon_social_oec ( id_oec, razon_social, estado, canton, provincia) VALUES (%s,%s,%s,%s,%s)'
for row in tqdm(df_oec.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla dim_perfil

In [ ]:
df_perfil = pd.read_csv(os.path.join(path_dm_oec,'dim_perfil.csv'))
df_perfil['id_perfil'] = df_perfil['id_perfil'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_perfil_oec")
mycursor.execute('''CREATE TABLE dim_perfil_oec (
id_perfil VARCHAR(4) PRIMARY KEY,
familia VARCHAR(255) NOT NULL,
sector VARCHAR(255) NOT NULL,
perfil VARCHAR(255) NOT NULL,
modalidad VARCHAR(63) NOT NULL,
carga_horaria INT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_perfil_oec (id_perfil, familia, sector, perfil, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_perfil.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla dim_fecha_oec

In [ ]:
df_fecha_oec = pd.read_csv(os.path.join(path_dm_oec,'dim_fecha.csv'))
df_fecha_oec['id_fecha'] = df_fecha_oec['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_fecha_oec")
mycursor.execute('''CREATE TABLE dim_fecha_oec (
id_fecha VARCHAR(4) PRIMARY KEY,
dia INT NOT NULL,
mes INT NOT NULL,
anio INT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha_oec (id_fecha, dia, mes, anio) VALUES (%s,%s,%s,%s)'
for row in tqdm(df_fecha_oec.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla fact_oec

In [ ]:
df_fact_oec = pd.read_csv(os.path.join(path_dm_oec,'fact_oec.csv'))
df_fact_oec['id_razon_social'] = df_fact_oec['id_razon_social'].astype(str)
df_fact_oec['id_perfil'] = df_fact_oec['id_perfil'].astype(str)
df_fact_oec['id_fecha'] = df_fact_oec['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS fact_oec")
mycursor.execute('''CREATE TABLE fact_oec (
id_razon_social VARCHAR(4) NOT NULL,
id_perfil VARCHAR(4) NOT NULL,
id_fecha VARCHAR(4) NOT NULL,
num_cer INT NOT NULL,
total_horas INT NOT NULL,
total_cursos INT NOT NULL,
volumen_capacitados INT NOT NULL,
total_ganancias FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO fact_oec(id_razon_social, id_perfil, id_fecha, num_cer, total_horas, total_cursos, volumen_capacitados, total_ganancias) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_fact_oec.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## CREAR CLAVES FORANEAS OEC

In [ ]:
mycursor.execute(''' 
ALTER TABLE fact_oec
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha_oec(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_oec
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_oec
ADD FOREIGN KEY (id_perfil) REFERENCES dim_perfil_oec(id_perfil)
''')


# DATAMART CI

In [ ]:
#os.listdir(path_dm_ci)

## Tabla dim_curso

In [ ]:
df_curso = pd.read_csv(os.path.join(path_dm_ci,'dim_curso_ci.csv'))
df_curso['id_curso'] = df_curso['id_curso'].astype(str)
df_curso = df_curso.fillna({'costo':0})
df_curso = df_curso.dropna()

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_curso_ci")
mycursor.execute('''CREATE TABLE dim_curso_ci (
id_curso VARCHAR(4) PRIMARY KEY,
curso VARCHAR(255) NOT NULL,
area VARCHAR(255) NOT NULL,
especialidad VARCHAR(255) NOT NULL,
modalidad VARCHAR(63) NOT NULL,
carga_horaria INT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_curso_ci (id_curso, curso, area, especialidad, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_curso.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla dim_fecha_ci

In [ ]:
df_fecha_ci = pd.read_csv(os.path.join(path_dm_ci,'dim_fecha_ci.csv'))
df_fecha_ci['id_fecha'] = df_fecha_ci['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS dim_fecha_ci")
mycursor.execute('''CREATE TABLE dim_fecha_ci (
id_fecha VARCHAR(4) PRIMARY KEY,
anio VARCHAR(15) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha_ci (id_fecha, anio) VALUES (%s,%s)'
for row in tqdm(df_fecha_ci.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

## Tabla fact_ci

In [ ]:
df_fact_ci = pd.read_csv(os.path.join(path_dm_ci,'fact_ci.csv'))
df_fact_ci['id_razon_social'] = df_fact_ci['id_razon_social'].astype(str)
df_fact_ci['id_curso'] = df_fact_ci['id_curso'].astype(str)
df_fact_ci['id_fecha'] = df_fact_ci['id_fecha'].astype(str)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS fact_ci")
mycursor.execute('''CREATE TABLE fact_ci (
id_razon_social VARCHAR(4) NOT NULL,
id_curso VARCHAR(4) NOT NULL,
id_fecha VARCHAR(4) NOT NULL,
num_cap INT NOT NULL,
total_cursos INT NOT NULL,
volumen_capacitados INT NOT NULL,
total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO fact_ci(id_razon_social, id_curso, id_fecha, num_cap, total_cursos, volumen_capacitados, total_horas, total_ganancias) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_fact_ci.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

In [ ]:
#df_ci.info()
# ', '.join(df_fact_ci.columns)

## CREAR CLAVES FORANEAS CI

In [ ]:
mycursor.execute(''' 
ALTER TABLE fact_ci
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha_ci(id_fecha)
''')

In [ ]:
mycursor.execute(''' 
ALTER TABLE fact_ci
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')

In [ ]:
mycursor.execute(''' 
ALTER TABLE fact_ci
ADD FOREIGN KEY (id_curso) REFERENCES dim_curso_ci(id_curso)
''')